In [ ]:
#11.6.1.2. 条件不佳的问题
!pip install d2l
%matplotlib inline
import torch
from d2l import torch as d2l

eta = 0.4
def f_2d(x1, x2):
    return 0.1 * x1 ** 2 + 2 * x2 ** 2
def gd_2d(x1, x2, s1, s2):
    return (x1 - eta * 0.2 * x1, x2 - eta * 4 * x2, 0, 0)

d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))

In [ ]:
eta = 0.6
d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))

In [ ]:
#11.6.1.3. 动量方法
#请注意，对于  β=0 ，我们恢复常规梯度下降。在深入研究数学属性之前，让我们快速看一下算法在实践中的行为方式。
def momentum_2d(x1, x2, v1, v2):
    v1 = beta * v1 + 0.2 * x1
    v2 = beta * v2 + 4 * x2
    return x1 - eta * v1, x2 - eta * v2, v1, v2

eta, beta = 0.6, 0.5
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

In [ ]:
#正如我们所看到的那样，即使我们以前使用的学习率相同，势头仍然很好地收敛。让我们看看当我们降低动量参数时会发生什么。
#将其减半至  β=0.25  会导致一条几乎没有收敛的轨迹。
#尽管如此，它比没有动力要好得多（当解决方案分歧时）。
eta, beta = 0.6, 0.25
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

In [ ]:
#11.6.1.4. 有效样品重量
d2l.set_figsize()
betas = [0.95, 0.9, 0.6, 0]
for beta in betas:
    x = torch.arange(40).detach().numpy()
    d2l.plt.plot(x, beta ** x, label=f'beta = {beta:.2f}')
d2l.plt.xlabel('time')
d2l.plt.legend();

In [ ]:
#11.6.2.1. 从头开始实施
#与（迷你匹配）随机梯度下降相比，动量方法需要维持一组辅助变量，即速度。
#它与梯度（以及优化问题的变量）具有相同的形状。在下面的实施中，我们称之为这些变量 states。
def init_momentum_states(feature_dim):
    v_w = torch.zeros((feature_dim, 1))
    v_b = torch.zeros(1)
    return (v_w, v_b)

def sgd_momentum(params, states, hyperparams):
    for p, v in zip(params, states):
        with torch.no_grad():
            v[:] = hyperparams['momentum'] * v + p.grad
            p[:] -= hyperparams['lr'] * v
        p.grad.data.zero_()

In [ ]:
#让我们看看这在实践中是如何运作的。
def train_momentum(lr, momentum, num_epochs=2):
    d2l.train_ch11(sgd_momentum, init_momentum_states(feature_dim),
                   {'lr': lr, 'momentum': momentum}, data_iter,
                   feature_dim, num_epochs)

data_iter, feature_dim = d2l.get_data_ch11(batch_size=10)
train_momentum(0.02, 0.5)

In [ ]:
train_momentum(0.01, 0.9)

In [ ]:
#降低学习率进一步解决了任何非平滑优化问题的问题。将其设置为  0.005  会产生良好的收敛性能。
train_momentum(0.005, 0.9)

In [ ]:
#11.6.2.2. 简洁的实施
#自从标准 sgd 求解器已经建立了势头以来，在 Gluon 没什么可做的。设置匹配参数会产生非常类似的轨迹。
trainer = torch.optim.SGD
d2l.train_concise_ch11(trainer, {'lr': 0.005, 'momentum': 0.9}, data_iter)

In [ ]:
#11.6.3.2. 标量函数
lambdas = [0.1, 1, 10, 19]
eta = 0.1
d2l.set_figsize((6, 4))
for lam in lambdas:
    t = torch.arange(20).detach().numpy()
    d2l.plt.plot(t, (1 - eta * lam) ** t, label=f'lambda = {lam:.2f}')
d2l.plt.xlabel('time')
d2l.plt.legend();